In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [67]:
train_dataset = pd.read_csv('../datasets/train.csv')
test_dataset = pd.read_csv('../datasets/test.csv')

In [68]:
y = train_dataset['rh98']
X = train_dataset.drop(columns=['rh98'])

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [70]:
clf = RandomForestRegressor(n_estimators=100, max_depth=40, n_jobs=-1)
clf.fit(X_train, y_train)

RandomForestRegressor(max_depth=40, n_jobs=-1)

In [71]:
prediction = clf.predict(test_dataset)
rh98_series = pd.Series(prediction)

In [74]:
submission = pd.concat([test_dataset['id'], rh98_series], axis=1)
submission.set_axis(['id', 'rh98'], axis='columns')

,index,id,0
0,0,1,9.2095
1,1,11,10.7686
2,2,12,7.2055
3,3,13,7.2196
4,4,17,8.1642
...,...,...,...
7263,7263,36336,9.8815
7264,7264,36354,10.1663
7265,7265,36360,9.2593
7266,7266,36366,5.2238


In [75]:
submission.to_csv('pred.csv', index=False)